<a href="https://colab.research.google.com/github/bbmt/BSP_Reader/blob/main/FPA_ReaderBSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Script for extracting images from multiple FPA BSP Agilent files
Imbuia Beamline@LNLS

**This script works for FPA bsp files. This is not for multispectra, pontual files.**

**This script was not tested for MOSAIC files.**

**Please name your file without whitespaces.**

**! ! ! IMPORTANT: Before running the script, please set the paths.
Usually, /content/drive/Mydrive/CHOOSEFOLDER/ ! ! !**

For processing the file, hit **Crtl-F9** (or Command-F9 if you are using Mac). The script will extract all relevant data from the BSP file. The only required user action is in the *Upload file* cell.

*If any error occurs, please send a printscreen and the bsp file to bruno.torres@lnls.br or brunobassi@ifsc.usp.br.*

In [1]:
import os
import olefile
import re
import glob

In [9]:
#@title Folder PATH - **ACTION REQUIRED**
#@markdown Set the path to folder with multiple bps files

path = '/home/bruno/PreguicaGigante/data/iii/' # remmember to add '/' at the end

file_pattern = "*.bsp" #you may change the pattern
files_path = sorted(glob.glob(path + file_pattern))
print(files_path) #sanity check

['/home/bruno/PreguicaGigante/data/iii/2iii_4_27b_borda_1.bsp', '/home/bruno/PreguicaGigante/data/iii/2iii_4_27b_borda_2.bsp', '/home/bruno/PreguicaGigante/data/iii/2iii_4_27b_meio_1.bsp', '/home/bruno/PreguicaGigante/data/iii/2iii_4_27b_meio_2.bsp', '/home/bruno/PreguicaGigante/data/iii/4_iii_25c_borda_1.bsp', '/home/bruno/PreguicaGigante/data/iii/4_iii_25c_borda_2.bsp', '/home/bruno/PreguicaGigante/data/iii/4_iii_25c_meio_1.bsp', '/home/bruno/PreguicaGigante/data/iii/5a_iii4_24b_borda_1.bsp', '/home/bruno/PreguicaGigante/data/iii/5a_iii4_24b_borda_2.bsp', '/home/bruno/PreguicaGigante/data/iii/5a_iii4_24b_meio_1.bsp', '/home/bruno/PreguicaGigante/data/iii/iii4_26c_7c.bsp', '/home/bruno/PreguicaGigante/data/iii/iii4_26c_7c_borda_2.bsp', '/home/bruno/PreguicaGigante/data/iii/iii4_26c_7c_meio.bsp', '/home/bruno/PreguicaGigante/data/iii/iii4_26c_7c_meio_2.bsp', '/home/bruno/PreguicaGigante/data/iii/iii_23a_304_311_fr11_q1_1_borda.bsp', '/home/bruno/PreguicaGigante/data/iii/iii_23a_304_311

In [11]:
#@title Folder Structure

path_to_save = '/home/bruno/PreguicaGigante/data/iii' #You may change the save path
os.makedirs(path_to_save+'/Images',exist_ok=True)
os.makedirs(path_to_save+'/TempBin',exist_ok=True)

In [12]:
print('Processing bsp files...')

def retrieve_image(filename):

    if olefile.isOleFile(filename):
        with olefile.OleFileIO(filename) as ole:
            streams = ole.listdir(storages=False, streams=True)

            streams_list = ole.openstream(['Spectra','IndexTable']).read()

            streams_list = re.findall(b'(\\w{8}-\\w{8})',streams_list)

    #  with olefile.OleFileIO(filename) as ole:
            stream_data = []
            for stream_ in streams_list:
                stream_data.append(ole.openstream(['Spectra',stream_.decode()]).read())

    for name,data in zip(streams_list,stream_data):
        with open(path_to_save+'/TempBin/'+name.decode()+'.bin','wb+') as f:
            f.write(data)

    for stream_name in streams_list:

        jpg_byte_start = b'\xff\xd8\xff\xe0'
        jpg_byte_end = b'\xff\xd9'
        jpg_image = bytearray()

        with open(path_to_save+'/TempBin/'+stream_name.decode()+'.bin', 'rb') as f:
            full_extracted_data = f.read()

            startimg = full_extracted_data.find(jpg_byte_start)

            if startimg == -1:
                print('!Warning!Could not find a valid JPEG image in the data stream:'+stream_name.decode())
                print('Filename:'+filename.split('/')[-1].split(',')[0])
                print('It is ok if you did not take a visible image for this experiment.\n')

            else:
                print('Image found:'+stream_name.decode()+' Saving...\n')
                endimg = full_extracted_data.find(jpg_byte_end, startimg) + len(jpg_byte_end)
                jpg_image += full_extracted_data[startimg:endimg]

                with open(path_to_save+'/Images/'+filename.split('/')[-1].split(',')[0]+'.jpg', 'wb+') as f:
                    f.write(jpg_image)

for file in files_path:
    retrieve_image(file)

Processing bsp files...
Image found:00025823-674FA25E Saving...

Image found:00025825-674FA42B Saving...

Image found:00025826-674FA5E0 Saving...

Image found:00025827-674FA7B4 Saving...

Image found:00003799-674FBB39 Saving...

Image found:0000379A-674FBD6E Saving...

Image found:0000379B-674FBFE2 Saving...

Image found:000037A4-674FCC25 Saving...

Image found:000037A5-674FCE56 Saving...

Image found:000037A6-674FD04A Saving...

Image found:00020DEC-674E176D Saving...

Image found:00020DEF-674E1C3F Saving...

Image found:00020DEE-674E1A0A Saving...

Image found:00020DF0-674E1EDA Saving...

Image found:0003317D-674E7A34 Saving...

Image found:00020E0F-674E70CF Saving...

Image found:0003317F-674E7CFA Saving...

Image found:0003317B-674E774F Saving...

Image found:0004BE02-674F8A0D Saving...

Image found:00049EA8-674F821F Saving...

Image found:00049EA9-674F8468 Saving...

Image found:00049EAB-674F86AF Saving...

Image found:00028C36-674F0D1A Saving...

Image found:00049E9D-674F6CC2 Sav

In [13]:
#@title Cleaning up and saving files to zip...

for temp_bin in os.listdir(path_to_save+'/TempBin/'):
    os.unlink(path_to_save+'/TempBin/'+temp_bin)
os.rmdir(path_to_save+'/TempBin')